In [2]:
states = ('Healthy', 'Fever')
end_state = 'E'
 
observations = ('normal', 'cold', 'dizzy')
 
start_probability = {'Healthy': 0.6, 'Fever': 0.4}
 
transition_probability = {
   'Healthy' : {'Healthy': 0.69, 'Fever': 0.3, 'E': 0.01},
   'Fever' : {'Healthy': 0.4, 'Fever': 0.59, 'E': 0.01},
   }
 
emission_probability = {
   'Healthy' : {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
   'Fever' : {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6},
   }

In [14]:
def fwd_bkw(x, states, a_0, a, e, end_st):
    L = len(x)
 
    fwd = []
    f_prev = {}
    # forward part of the algorithm
    for i, x_i in enumerate(x):
        f_curr = {}
        for st in states:
            if i == 0:
                # base case for the forward part
                prev_f_sum = a_0[st]
            else:
                prev_f_sum = sum(f_prev[k]*a[k][st] for k in states)
 
            f_curr[st] = e[st][x_i] * prev_f_sum
 
        fwd.append(f_curr)
        f_prev = f_curr
 
    p_fwd = sum(f_curr[k]*a[k][end_st] for k in states)
 
    bkw = []
    b_prev = {}
    # backward part of the algorithm
    for i, x_i_plus in enumerate(reversed(x[1:]+(None,))):
        print x_i_plus
        b_curr = {}
        for st in states:
            if i == 0:
                # base case for backward part
#                 print 'base case: ', st, end_st
                b_curr[st] = a[st][end_st]
            else:
                b_curr[st] = sum(a[st][l]*e[l][x_i_plus]*b_prev[l] for l in states)
 
        bkw.insert(0,b_curr)
        b_prev = b_curr
 
    p_bkw = sum(a_0[l] * e[l][x[0]] * b_curr[l] for l in states)
#     print len(bkw)
 
    # merging the two parts
    posterior = []
    for i in range(L):
        posterior.append({st: fwd[i][st]*bkw[i][st]/p_fwd for st in states})
 
    assert p_fwd == p_bkw
    return fwd, bkw, posterior

In [19]:
def example():
    return fwd_bkw(observations,
                   states,
                   start_probability,
                   transition_probability,
                   emission_probability,
                   end_state)
 
fwd, bkw, posterior = example()
print posterior

None
dizzy
cold
[{'Healthy': 0.8770110375573259, 'Fever': 0.1229889624426741}, {'Healthy': 0.623228030950954, 'Fever': 0.3767719690490461}, {'Healthy': 0.2109527048413057, 'Fever': 0.7890472951586943}]


In [23]:
# # potential spell-check version??

# # Modified from wikipedia/AM207 lecture notes
# def fwd_bkw(words, dictionary, longest_word_length, start_prob, default_start_prob, \
#                 transition_prob, default_transition_prob, end_prob, default_end_prob, \
#                 num_word_suggestions=5000):
    
#     # FOR TESTING - DELETE EVENTUALLY
#     if type(words) != list:
#         words = re.findall('[a-z]+', words.lower())  # separate by words by non-alphabetical characters

#     path_word = []
    
#     L = len(words)

#     # forward part of the algorithm
    
#     fwd = []
#     f_prev = {}
    
#     for i, x_i in enumerate(words):

#         # Character level correction
#         corrections = get_suggestions(x_i, dictionary, longest_word_length, silent=True, min_count=1)

#         if len(corrections) > num_word_suggestions:
#             corrections = corrections[0:num_word_suggestions]
#         if len(corrections) > 0:
#             path_word.append(corrections[0][0])  # string of most frequent word tuple
        
#         f_curr = {}
        
#         for st in corrections:
#             if i == 0:
#                 # base case for the forward part
#                 prev_f_sum = get_start_prob(st, start_prob, default_start_prob)
#             else:
#                 prev_f_sum = math.log(sum(math.exp(f_prev[k] \
#                                  + get_transition_prob(st, k, transition_prob, default_transition_prob)) \
#                                     for k in prev_corrections))
 
#             f_curr[st[0]] = get_emission_prob(dameraulevenshtein(st, x_i)) + prev_f_sum
 
#         fwd.append(f_curr)
#         f_prev = f_curr
        
#         prev_corrections = [j[0] for j in corrections]
 
#     p_fwd = math.log(sum(math.exp(f_curr[k[0]] \
#                 + get_transition_prob(words[-1], k[0], transition_prob, default_transition_prob))
#                     for k in corrections))

#     # backward part of the algorithm

#     bkw = []
#     b_prev = {}

#     for i, x_i_plus in enumerate(reversed(words[1:] + [None])):
#         print i, x_i_plus
        
#         if i == 0:
#             corrections = get_suggestions(words[-1], dictionary, longest_word_length, silent=True, min_count=1)
#         else:
#             corrections = get_suggestions(x_i_plus, dictionary, longest_word_length, silent=True, min_count=1)

#         b_curr = {}
        
#         for st in corrections:
#             if i == 0:
# #                 base case for backward part
#                 b_curr[st[0]] = get_end_prob(st, end_prob, default_end_prob)
#             else:
#                 b_curr[st[0]] = 0
# #                 b_curr[st[0]] = math.log(sum(math.exp(get_transition_prob(l, st[0], transition_prob, default_transition_prob) \
# #                                  + get_emission_prob(dameraulevenshtein(l, x_i_plus)) \
# #                                  + b_prev[l])
# #                                         for l in prev_corrections))
 
#         bkw.insert(0,b_curr)
#         b_prev = b_curr
        
#         prev_corrections = [j[0] for j in corrections]
 
# #     p_bkw = math.log(sum(math.exp(get_start_prob(l, start_prob, default_start_prob) \
# #                 + get_emission_prob(dameraulevenshtein(l, x[0])) \
# #                 + b_curr[l]) \
# #                     for l in states))
 
#     # merging the two parts
#     posterior = []
#     for i in range(L):
#         states = list(set(fwd[i].keys() + bkw[i].keys()))
#         posterior.append({st: (get_val(fwd[i], st) + get_val(bkw[i], st))/p_fwd for st in states})
 
#     map_posterior = [max(p, key=lambda i: p[i]) for p in posterior]

# #     assert p_fwd == p_bkw
#     return path_word, map_posterior